**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [19]:
!pip install keras-flops

In [153]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils
from keras_flops import get_flops

In [21]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [22]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [23]:
# Преобразуйте метки классов в one_hot формат
y_train = np_utils.to_categorical(y_train, CLASSES)
y_val = np_utils.to_categorical(y_val, CLASSES)

In [24]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [236]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, kernel_size=3, strides = 2, padding='same', input_shape=(32,32,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
    tf.keras.layers.Activation('relu'),
    
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dropout(0.2),
    
    
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
    
    
])

In [237]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-26 00:49:03.169077: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 00:49:03.169452: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-26 00:49:03.169563: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-26 00:49:03.170013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 00:49:03.170382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 00:49:03.170682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

le:
node name | # float_ops
_TFProfRoot (--/443.06k flops)
  sequential_63/conv2d_91/Conv2D (221.18k/221.18k flops)
  sequential_63/dense_66/MatMul (204.80k/204.80k flops)
  sequential_63/batch_normalization_96/FusedBatchNormV3 (8.29k/8.29k flops)
  sequential_63/conv2d_91/BiasAdd (4.10k/4.10k flops)
  sequential_63/max_pooling2d_79/MaxPool (4.10k/4.10k flops)
  sequential_63/activation_157/Softmax (500/500 flops)
  sequential_63/dense_66/BiasAdd (100/100 flops)

======================End of Report==========================


In [238]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 16, 16, 16)        448       
_________________________________________________________________
batch_normalization_96 (Batc (None, 16, 16, 16)        64        
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 8, 8, 16)          0         
_________________________________________________________________
activation_156 (Activation)  (None, 8, 8, 16)          0         
_________________________________________________________________
flatten_59 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dropout_58 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_66 (Dense)             (None, 100)             

In [249]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=2000, decay_rate=0.25)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [250]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=40
)

Epoch 1/40
391/391 [==============================] - 2s 5ms/step - loss: 2.8984 - accuracy: 0.3051 - val_loss: 3.1108 - val_accuracy: 0.2698
Epoch 2/40
391/391 [==============================] - 1s 3ms/step - loss: 2.7089 - accuracy: 0.3417 - val_loss: 2.9783 - val_accuracy: 0.2848
Epoch 3/40
391/391 [==============================] - 1s 3ms/step - loss: 2.5970 - accuracy: 0.3619 - val_loss: 2.8363 - val_accuracy: 0.3245
Epoch 4/40
391/391 [==============================] - 1s 4ms/step - loss: 2.5111 - accuracy: 0.3775 - val_loss: 2.8760 - val_accuracy: 0.3216
Epoch 5/40
391/391 [==============================] - 1s 3ms/step - loss: 2.4401 - accuracy: 0.3937 - val_loss: 2.8244 - val_accuracy: 0.3252
Epoch 6/40
391/391 [==============================] - 1s 3ms/step - loss: 2.3988 - accuracy: 0.4002 - val_loss: 2.7917 - val_accuracy: 0.3298
Epoch 7/40
391/391 [==============================] - 2s 5ms/step - loss: 2.3521 - accuracy: 0.4087 - val_loss: 2.7748 - val_accuracy: 0.3365
Epoch 

In [251]:
scores = model.evaluate(X_val, y_val)
print(f"Final accuracy: {round((scores[1]*100),2)}%")

313/313 [==============================] - 1s 2ms/step - loss: 2.7492 - accuracy: 0.3452
Final accuracy: 34.52%
